## Przygotowanie


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

keras = tf.keras

In [ ]:
def plot_series(time, series, format="-", start=0, end=None, label=None):
    plt.plot(time[start:end], series[start:end], format, label=label)
    plt.xlabel("Time")
    plt.ylabel("Value")
    if label:
        plt.legend(fontsize=14)
    plt.grid(True)


#Występowanie
Szeregi czasowe występują dosłownie w każdej dziedzinie życia. Ich zastosowanie można łatwo dostrzec m.in w prognozach pogody, na giełdzie, w historycznych danych, pomiarach.

#Szereg czasowy
Szeregiem czasowym nazywamy uporządkowaną sekwencję, w której dane ułożone są sekwencyjnie, a kluczem określającym ich położenie jest czas.
W zależności od wartości określanych przez encjęw danym punkcie w czasie, dane możemy podzielić na:
- jednowymiarowe (jedna wartość w jednym punkcie)
- wielowymiarowe (wiele wartości w jednym punkcie)

#Przykładowy zbiór danych jednowymiarowych.
Zbiór szeregów przedstawiających ilość sprzedanych szamponów na przestrzeni 3 lat

In [ ]:
url = "https://raw.githubusercontent.com/Mervolt/TimeSeriesTutorial/master/shampoo.csv"

shampoo_dataset = pd.read_csv(url, error_bad_lines=False)
print(shampoo_dataset)

In [ ]:
time, values = shampoo_dataset["Month"], shampoo_dataset["Sales"]

plt.figure(figsize=(10, 6))
plot_series(time, values, label = False)
plt.show()

#Przykładowy zbiór danych wielowymiarowych
Zbiór szeregów przedstawiających obserwacje ilości ozonu przez 6 lat


In [ ]:
url = "https://raw.githubusercontent.com/Mervolt/TimeSeriesTutorial/master/ozone.csv"

ozone_dataset = pd.read_csv(url)
print(ozone_dataset)

time, values = ozone_dataset["Time"], ozone_dataset[ozone_dataset.columns[2:6]]

plt.figure(figsize=(10, 6))
plot_series(time, values["2"], label = False)
plot_series(time, values["3"], label = False)
plot_series(time, values["4"], label = False)
plot_series(time, values["5"], label = False)
plt.show()

#Wspólne cechy szeregów

Wiele szeregów czasowych posiada takie właściwości jak:
- trend (np. monotoniczny wzrost lub spadek)
- sezonowość, którą można zaobserwować jako okres na wykresie (np. ilość turystów w zależności od miesiąca pokazywać będzie największe wartości w okresie letnim)
- szum, czyli zakłócenia, drobne błędy wartości występujące w zbiorze danych

#Trend rosnący

In [ ]:
def trend(time, slope=0):
    return slope * time

time = np.arange(4 * 365 + 1)
baseline = 10
series = baseline + trend(time, 0.1)

plt.figure(figsize=(10, 6))
plot_series(time, series)
plt.show()

#Sezonowość

In [ ]:
def seasonal_pattern(season_time):
    """Just an arbitrary pattern, you can change it if you wish"""
    return np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))

def seasonality(time, period, amplitude=1, phase=0):
    """Repeats the same pattern at each period"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)

amplitude = 40
series = seasonality(time, period=365, amplitude=amplitude)

plt.figure(figsize=(10, 6))
plot_series(time, series)
plt.show()

#Szum

In [ ]:
def white_noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

noise_level = 5
noise = white_noise(time, noise_level, seed=42)

plt.figure(figsize=(10, 6))
plot_series(time, noise)
plt.show()

#Przewidywanie

Posiadając zbiór przedstawiający szereg czasowy pewnych wartości możemy zabrać się za próby oszacowania wartości jakie wystąpią w przyszłości.

Użyjemy zbioru przedstawiającego minimalne temperature w Melbourne w latach 1981-1990.

Zbiór podzielimy zgodnie z metodą "Fixed partitioning na część do uczenia, walidacyjną i testującą.

#Naiwne przewidywanie

Najprostszym możliwym sposobem jest po prostu branie poprzedniej wartości jako przewidywaną. Metoda wydaje się być prymitywna, ale osiągane przez nią rezultaty są warte zaobserwowania chociażby dla celów porównania z innymi metodami.



In [ ]:
url = "https://raw.githubusercontent.com/Mervolt/TimeSeriesTutorial/master/melbourne_min_temp.csv"

dataset = pd.read_csv(url)
print(dataset)

In [ ]:
split = 3000
time, values = dataset["Date"], dataset["Temp"]
x_train, y_train = time[:split], values[:split]
x_valid, y_valid = time[split:], values[split:] 

In [ ]:
naive_forecast = values[split - 1:-1]

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(x_valid, y_valid, label="Values")
plot_series(x_valid, naive_forecast, label="Forecast")

Wykresy nachodzą na siebie w takim stopniu, że nie można ich od siebie odróżnić. Wydzielimy teraz dla celów demonstracyjnych podzbiór.

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(x_valid, y_valid, start=0, end=150, label="Values")
plot_series(x_valid, naive_forecast, start=1, end=151, label="Forecast")

Możemy zaobserwować, że przewidywania są po prostu 1 krok za rzeczywistymi wartościami.

W celu ewaluacji potrzebujemy metryk. Użyjemy w tym przypadku metryk średniokwadratowej oraz odległości w przestrzeni Euklidesowej.

In [ ]:
print(keras.metrics.mean_absolute_error(y_valid, naive_forecast).numpy())
print(keras.metrics.mean_squared_error(y_valid, naive_forecast).numpy())

#Ruchoma średnia
Podejściem, które teraz zostanie zaprezentowane to ruchoma średnia. Jest ono relatywnie proste i polega na wyciąganiu średniej z okresu o długości n. To "ruchome okno" o długości n przesuwamy po całym zbiorze danych.
Przykładowo dla okna o długości, wartość pola o indeksie 7 liczymy jako średnią z pól o indeksach 5, 6 i 7, a dla pola o indeksie 22 z pól o indeksach 20, 21 i 22.

Zalety:
- redukuje szum

Wady:
- nie uwzględnia sezonowości
- nie uwzględnia trendów

In [ ]:
def moving_average_forecast(values, window_size):
  forecast = []
  for time in range(len(values) - window_size):
    forecast.append(values[time:time + window_size].mean())
  return np.array(forecast)

In [ ]:
moving_avg = moving_average_forecast(values, 3)[split - 3:]

plt.figure(figsize=(10, 6))
plot_series(x_valid, y_valid, label="Values")
plot_series(x_valid, moving_avg, label="Ruchoma średnia (3 dni)")

In [ ]:
print(keras.metrics.mean_absolute_error(y_valid, moving_avg).numpy())
print(keras.metrics.mean_squared_error(y_valid, moving_avg).numpy())

Szok! Osiągneliśmy gorsze wyniki niż w przypadku naiwnego podejścia. Może długość aplikowanego okna była za mała?
Spróbujmy dla innej wielkości okna.

In [ ]:
moving_avg = moving_average_forecast(values, 10)[split - 10:]

plt.figure(figsize=(10, 6))
plot_series(x_valid, y_valid, label="Values")
plot_series(x_valid, moving_avg, label="Ruchoma średnia (10 dni)")

In [ ]:
print(keras.metrics.mean_absolute_error(y_valid, moving_avg).numpy())
print(keras.metrics.mean_squared_error(y_valid, moving_avg).numpy())

Wciąż gorsze wyniki. Powodem jest tutaj potężna wada tego podejścia, a mianowicie brak uwzględnienia sezonowości, która w przypadku temperatur jest oczywista i bardzo wyraźna

#Ulepszona metoda ruchomej średniej
Aby ulepszyć metodę ruchomej średniej należy zlikwidować jej wady - brak uwzględniania trendów oraz brak uwzględniania sezonowości.
W tym celu należy specjalnie zadaptować nasz zbiór danych. Zamiast korzystać po prostu z naszego zbioru, korzystać będziemy z różnic (wartość - wartość wcześniejsza o pewien czas t, np. 1 rok i różnica = czerwiec 1984 - czerwiec 1983 )

In [ ]:
diff_values = (values[365:].reset_index() - values[:-365].reset_index())['Temp']
diff_time = time[365:]
plt.figure(figsize=(10, 6))
plot_series(diff_time, diff_values, label="Values(t) – Values(t–365)")
plt.show()

Podzbiór walidacyjny

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(x_valid, diff_values[split - 365:], label="Values(t) – Values(t–365)")
plt.show()

In [ ]:
diff_moving_avg = moving_average_forecast(diff_values, 30)[split - 365 - 30:]

plt.figure(figsize=(10, 6))
plot_series(x_valid, diff_values[split - 365:], label="Values(t) – Values(t–365)")
plot_series(x_valid, diff_moving_avg, label="Moving Average of Diff")
plt.show()

W porządku. Obliczyliśmy ruchomą średnią, ale nie jest to przecież nasz zbiór danych, a to on jest dla nas interesujący. W takim razie musimy go odzyskać.
Aby to zrobić należy dodać wartości z przeszłości.

In [ ]:
diff_moving_avg_plus_past = values[split - 365:-365] + diff_moving_avg

plt.figure(figsize=(10, 6))
plot_series(x_valid, y_valid, label="Values")
plot_series(x_valid, diff_moving_avg_plus_past, label="Forecasts")
plt.show()

In [ ]:
print(keras.metrics.mean_absolute_error(y_valid, diff_moving_avg_plus_past).numpy())
print(keras.metrics.mean_squared_error(y_valid, diff_moving_avg_plus_past).numpy())

Otrzymaliśmy tragiczne wyniki. Spróbujmy zredukować szum w początkowych danych.

In [ ]:
diff_moving_avg_plus_smooth_past = moving_average_forecast(values[split - 370:-359], 11) + diff_moving_avg

plt.figure(figsize=(10, 6))
plot_series(x_valid, y_valid, label="Values")
plot_series(x_valid, diff_moving_avg_plus_smooth_past, label="Forecasts")
plt.show()

In [ ]:
print(keras.metrics.mean_absolute_error(y_valid, diff_moving_avg_plus_smooth_past).numpy())
print(keras.metrics.mean_squared_error(y_valid, diff_moving_avg_plus_smooth_past).numpy())

Znacznie lepiej, jednakże wciąż otrzymaliśmy gorsze rezultaty niż w przypadku naiwnego przewidywania

#Przewidywanie - Machine Learning
Użyjemy sieci o warstwie 1 bez żadnej funkcji aktywacji. Użyjemy metody regresji.

In [ ]:
def window_dataset(series, window_size, batch_size=32,
                   shuffle_buffer=1000):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = window_dataset(y_train, window_size)
valid_set = window_dataset(y_valid, window_size)

model = keras.models.Sequential([
  keras.layers.Dense(1, input_shape=[window_size])
])
optimizer = keras.optimizers.SGD(lr=1e-5, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae", "mse"])
model.fit(train_set, epochs=100, validation_data=valid_set)

Sukces! Udało nam się przebić naiwne przewidywanie na danych testowych.

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = window_dataset(y_train, window_size)

model = keras.models.Sequential([
  keras.layers.Dense(1, input_shape=[window_size])
])

lr_schedule = keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-6 * 10**(epoch / 30))
optimizer = keras.optimizers.SGD(lr=1e-6, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])

In [ ]:
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-6, 1e-3, 0, 20])

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = window_dataset(y_train, window_size)
valid_set = window_dataset(y_valid, window_size)

model = keras.models.Sequential([
  keras.layers.Dense(1, input_shape=[window_size])
])
optimizer = keras.optimizers.SGD(lr=1e-5, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae", "mse"])
early_stopping = keras.callbacks.EarlyStopping(patience=10)
model.fit(train_set, epochs=500,
          validation_data=valid_set,
          callbacks=[early_stopping])

In [ ]:
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [ ]:
lin_forecast = model_forecast(model, values[split - window_size:-1], window_size)[:, 0]
lin_forecast.shape

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(x_valid, y_valid)
plot_series(x_valid, lin_forecast)

In [ ]:
print(keras.metrics.mean_absolute_error(y_valid, lin_forecast).numpy())
print(keras.metrics.mean_squared_error(y_valid, lin_forecast).numpy())

Rezultat z dość znaczną poprawą :)

Dwuwarstwowy model z funkcją aktywacji relu.

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = window_dataset(y_train, window_size)

model = keras.models.Sequential([
  keras.layers.Dense(10, activation="relu", input_shape=[window_size]),
  keras.layers.Dense(10, activation="relu"),
  keras.layers.Dense(1)
])

lr_schedule = keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-7 * 10**(epoch / 20))
optimizer = keras.optimizers.SGD(lr=1e-7, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae", "mse"])
history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])

In [ ]:
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-7, 5e-3, 0, 30])

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = window_dataset(y_train, window_size)
valid_set = window_dataset(y_valid, window_size)

model = keras.models.Sequential([
  keras.layers.Dense(10, activation="relu", input_shape=[window_size]),
  keras.layers.Dense(10, activation="relu"),
  keras.layers.Dense(1)
])

optimizer = keras.optimizers.SGD(lr=1e-5, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae", "mse"])
early_stopping = keras.callbacks.EarlyStopping(patience=10)
model.fit(train_set, epochs=500,
          validation_data=valid_set,
          callbacks=[early_stopping])

In [ ]:
dense_forecast = model_forecast(
    model,
    values[split - window_size:-1],
    window_size)[:, 0]

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(x_valid, y_valid)
plot_series(x_valid, dense_forecast)

In [ ]:
print(keras.metrics.mean_absolute_error(y_valid, dense_forecast).numpy())
print(keras.metrics.mean_squared_error(y_valid, dense_forecast).numpy())

#RNN

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = window_dataset(y_train, window_size, batch_size=128)

model = keras.models.Sequential([
  keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                      input_shape=[None]),
  keras.layers.SimpleRNN(100, return_sequences=True),
  keras.layers.SimpleRNN(100),
  keras.layers.Dense(1),
  keras.layers.Lambda(lambda x: x * 200.0)
])
lr_schedule = keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-7 * 10**(epoch / 20))
optimizer = keras.optimizers.SGD(lr=1e-7, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae", "mse"])
history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])

In [ ]:
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-7, 1e-4, 0, 30])

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = window_dataset(y_train, window_size, batch_size=128)
valid_set = window_dataset(y_valid, window_size, batch_size=128)

model = keras.models.Sequential([
  keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                      input_shape=[None]),
  keras.layers.SimpleRNN(100, return_sequences=True),
  keras.layers.SimpleRNN(100),
  keras.layers.Dense(1),
  keras.layers.Lambda(lambda x: x * 200.0)
])
optimizer = keras.optimizers.SGD(lr=1.5e-6, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
early_stopping = keras.callbacks.EarlyStopping(patience=50)
model_checkpoint = keras.callbacks.ModelCheckpoint(
    "my_checkpoint", save_best_only=True)
model.fit(train_set, epochs=500,
          validation_data=valid_set,
          callbacks=[early_stopping, model_checkpoint])

In [ ]:
rnn_forecast = model_forecast(
    model,
    values[split - window_size:-1],
    window_size)[:, 0]

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(x_valid, y_valid)
plot_series(x_valid, rnn_forecast)

In [ ]:
print(keras.metrics.mean_absolute_error(y_valid, rnn_forecast).numpy())
print(keras.metrics.mean_squared_error(y_valid, rnn_forecast).numpy())